In [ ]:
import argparse
import os
from data_utils.ShapeNetDataLoader import PartNormalDataset
import torch
import logging
import sys
import importlib
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml  

In [ ]:
# 读取YAML文件  
with open('config.yaml', 'r', encoding='utf-8') as file:  
    config = yaml.safe_load(file)  
  
# 提取字段  
ROOT_DIR = config['project_base_dir']  
ShapeNet_path = config['dataset']['ShapeNet_path'] 

sys.path.append(os.path.join(ROOT_DIR, 'models'))

# 定义一个字典，映射每个类别名称到其对应的标签列表
seg_classes = {
    'Earphone': [16, 17, 18], 
    'Motorbike': [30, 31, 32, 33, 34, 35], 
    'Rocket': [41, 42, 43],
    'Car': [8, 9, 10, 11], 
    'Laptop': [28, 29], 
    'Cap': [6, 7], 
    'Skateboard': [44, 45, 46], 
    'Mug': [36, 37],
    'Guitar': [19, 20, 21], 
    'Bag': [4, 5], 
    'Lamp': [24, 25, 26, 27], 
    'Table': [47, 48, 49],
    'Airplane': [0, 1, 2, 3], 
    'Pistol': [38, 39, 40], 
    'Chair': [12, 13, 14, 15], 
    'Knife': [22, 23]
}

# 创建一个字典，将每个标签映射到对应的类别名称
seg_label_to_cat = {}  # 格式为 {标签: 类别名称}
for cat in seg_classes.keys():  # 遍历每个类别名称
    for label in seg_classes[cat]:  # 遍历每个类别下的标签
        seg_label_to_cat[label] = cat  # 将标签映射到类别名称
        


In [3]:
def to_categorical(y, num_classes):
    """将标签转换为1-hot编码"""
    # print(f"y: {y}")
    # print(f"num_classes: {num_classes}")
    # 创建一个大小为 (num_classes, num_classes) 的单位矩阵，每一行对应一个类别的1-hot编码
    new_y = torch.eye(num_classes)[y.cpu().data.numpy(),]
    
    # 如果输入张量 y 在 GPU 上，则将生成的1-hot编码张量也转移到 GPU 上
    if (y.is_cuda):
        return new_y.cuda()
    
    # 如果输入张量 y 在 CPU 上，则返回 CPU 上的1-hot编码张量
    return new_y

In [4]:
def generate_random_point_cloud(batch_size=1, num_points=2048, 
                                num_features=6, num_classes=40):
    """
    生成一组随机的点云数据、类别标签和分割标签。

    Parameters:
    - batch_size: 批量大小
    - num_points: 点云中的点数
    - num_features: 每个点的特征数
    - num_classes: 类别数量

    Returns:
    - points: 点云数据，形状为 [batch_size, num_points, num_features]
    - label: 类别标签，形状为 [batch_size]
    - target: 分割标签，形状为 [batch_size, num_points]
    """
    # 随机生成点云数据
    points = np.random.rand(batch_size, num_points, num_features).astype(np.float32)
    
    # 随机生成类别标签
    label = np.random.randint(0, num_classes, size=(batch_size,)).astype(np.int32)
    
    # 随机生成分割标签
    target = np.random.randint(0, 3, size=(batch_size, num_points)).astype(np.int32)
    
    # 转换为 PyTorch 张量
    points = torch.from_numpy(points)
    label = torch.from_numpy(label)
    target = torch.from_numpy(target)
    
    return points, label, target

In [5]:
def plot_3d_points(points: torch.Tensor, labels: np.ndarray):
    """
    绘制带有标签的三维点云图。

    参数:
    - points: torch.Tensor, 形状为(N, 3)的三维坐标。
    - labels: np.ndarray, 形状为(N,)的标签数组。
    """
    # 确保points是numpy数组
    points_np = points.numpy()

    # 创建图形和3D坐标轴
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # 绘制三维散点图
    scatter = ax.scatter(points_np[:, 0], points_np[:, 1], points_np[:, 2], c=labels, cmap='viridis')

    # 添加颜色条
    cbar = plt.colorbar(scatter)
    cbar.set_label('Labels')

    # 设置坐标轴标签
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')

    # 显示图形
    plt.show()

In [6]:
def parse_args():
    '''参数解析'''
    parser = argparse.ArgumentParser('PointNet')  # 创建ArgumentParser对象，用于处理命令行参数
    parser.add_argument('--batch_size', type=int, default=24, help='测试时的批处理大小')  # 添加batch_size参数，默认为24
    parser.add_argument('--gpu', type=str, default='0', help='指定GPU设备')  # 添加gpu参数，用于指定使用的GPU设备
    parser.add_argument('--num_point', type=int, default=2048, help='点云中的点数')  # 添加num_point参数，默认为2048，用于指定点云中的点数
    parser.add_argument('--log_dir', type=str, required=True, help='实验的根目录')  # 添加log_dir参数，必须指定，用于指定实验日志的根目录
    parser.add_argument('--normal', action='store_true', default=False, help='是否使用法线信息')  # 添加normal参数，用于选择是否使用法线信息
    parser.add_argument('--num_votes', type=int, default=3, help='通过投票聚合分割分数的次数')  # 添加num_votes参数，默认为3，用于指定投票次数
    return parser.parse_args()  # 解析并返回命令行参数

In [7]:
def main(args):
    '''HYPER PARAMETER'''
    # 设置CUDA可见的设备（指定使用的GPU）, 根据命令行参数设置
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
    # 检查是否有可用的CUDA设备, 并设置计算设备（GPU或CPU）
    device = torch.device("cuda" if torch.cuda.is_available() and args.gpu != '-1' else "cpu")
    experiment_dir = 'log/part_seg/' + args.log_dir

   
    points, label, target = generate_random_point_cloud(batch_size=1, 
                                                        num_points=2048, 
                                                        num_features=6, 
                                                        num_classes=4)
    cur_batch_size, NUM_POINT, _ = points.size()
    print('cur_batch_size:', cur_batch_size)
    print('NUM_POINT:', NUM_POINT)
    print('target:', target)
    print('target:', target.size())
    from collections import Counter
    # 统计每个值出现的次数
    target_unique_values = torch.unique(target)             # 不同的值
    target_num_part = target_unique_values.numel()          # 不同值的数量

    label_unique_values = torch.unique(label)               # 不同的值
    label_num_unique_values = label_unique_values.numel()   # 不同值的数量

    
    num_classes = 16 # 数据集中有16个类别
    num_part = 50  # 总共有部件类别

    '''MODEL LOADING'''
    # 获取实验目录中 logs 文件夹下的模型名称（假设只有一个模型文件）
    model_name = os.listdir(experiment_dir + '/logs')[0].split('.')[0]
    print('Loading model: ' + model_name)
    # 动态导入模型模块
    MODEL = importlib.import_module(model_name)
    
    # 使用导入的模块创建分类器模型实例，传入参数为部件类别数量和是否使用法线信息
    classifier = MODEL.get_model(num_part, normal_channel=args.normal).to(device)
    
    # 加载保存的最佳模型检查点
    checkpoint = torch.load(str(experiment_dir) + '/checkpoints/best_model.pth')
    
    # 将检查点中的模型状态字典加载到当前模型实例中
    classifier.load_state_dict(checkpoint['model_state_dict'])
    

    with torch.no_grad():
        # 模型设置
        classifier = classifier.eval()  # 将模型设置为评估模式
        # 将点云数据、标签和目标转换为浮点型并移动到设备（如GPU）
        points, label, target = points.float().to(device), label.long().to(device), target.long().to(device)

        # 转置点云数据的维度以匹配模型输入的要求
        points = points.transpose(2, 1)

        # 初始化投票池，用于存储多个投票轮次的分割预测
        vote_pool = torch.zeros(target.size()[0], target.size()[1], num_part).to(device)

        # 多轮投票预测以增强稳定性
        for _ in range(args.num_votes):
            seg_pred, _ = classifier(points, to_categorical(label, num_classes))
            vote_pool += seg_pred  # 将每轮的预测累加到投票池中

        # 将投票池中的预测结果取平均，得到最终的预测结果
        seg_pred = vote_pool / args.num_votes

        # 将预测结果从 GPU 转移到 CPU，并转换为 NumPy 数组
        cur_pred_val = seg_pred.cpu().data.numpy()
        cur_pred_val_logits = cur_pred_val  # 保留未处理的逻辑回归结果

        # 初始化一个零矩阵用于存储最终的预测结果
        cur_pred_val = np.zeros((cur_batch_size, NUM_POINT)).astype(np.int32)

        # 将目标标签从 GPU 转移到 CPU，并转换为 NumPy 数组
        target = target.cpu().data.numpy()

        # 对每一个点云实例进行处理
        # for i in range(cur_batch_size):
        # 获取当前实例的类别
        cat = seg_label_to_cat[target[0, 0]]
        
        # 获取当前实例的预测逻辑回归结果
        logits = cur_pred_val_logits[0, :, :]
        print('logits:', logits.shape)

        # 对该实例进行预测，并根据类别的标签范围调整预测结果
        cur_pred_val[0, :] = np.argmax(logits[:, seg_classes[cat]], 1) + seg_classes[cat][0]
        # 打印预测的分割标签
        print(f"点云实例的分割标签:")
        print(cur_pred_val[0, :])
        print(len(cur_pred_val[0, :]))

        # 点云可视化
        plot_3d_points(points, cur_pred_val[0, :])

In [8]:

if __name__ == '__main__':
    args = parse_args()
    main(args)


usage: PointNet [-h] [--batch_size BATCH_SIZE] [--gpu GPU]
                [--num_point NUM_POINT] --log_dir LOG_DIR [--normal]
                [--num_votes NUM_VOTES]
PointNet: error: the following arguments are required: --log_dir


SystemExit: 2

In [2]:
import numpy as np
np.random.rand(1, 10, 3).astype(np.float32)

array([[[0.34445795, 0.5455173 , 0.6004367 ],
        [0.95220596, 0.4126707 , 0.390423  ],
        [0.30783132, 0.49503064, 0.7470529 ],
        [0.27689543, 0.01214461, 0.14316031],
        [0.38528582, 0.9303482 , 0.43746695],
        [0.23816772, 0.7911151 , 0.3294376 ],
        [0.83521247, 0.19806777, 0.4595417 ],
        [0.89621156, 0.20914352, 0.61107063],
        [0.75776774, 0.8570657 , 0.44315097],
        [0.8760802 , 0.189009  , 0.08362827]]], dtype=float32)